# ex1-Export A NetCDF Data As A Time Series of GeoTiff Images

The command-line tool of [gdal_translate](https://gdal.org/programs/gdal_translate.html#gdal-translate) provided by GDAL should be the most commonly used option for converting raster data between different formats, while [CDO](https://code.mpimet.mpg.de/projects/cdo/) is another excellent command line suite for manipulating and analysing climate data. 
This time we will apply them to export each time slice from a NetCDF data as a single GeoTiff image(s). we still use the monthly mean sea level pressure of [mslp.mon.mean.nc](http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis2.surface.html) as an example.

Before moving on, make sure you have GDAL and CDO installed on your computer or set up a new experimental environment using docker just like me. The following is my ***Dockerfile***.

In [ ]:
RG BASE_CONTAINER=andrejreznik/python-gdal:stable
FROM $BASE_CONTAINER

LABEL maintainer="chy@climsystems.com"
RUN apt-get update && apt-get install -y --no-install-recommends cdo && \
    rm -rf /var/lib/apt/lists/*

## Check data information

We can use [***gdalinfo***] or [***cdo info***]. Here we use gdalinfo as we mainly use gdal_translate to convert data formats.

In [ ]:
gdalinfo mslp.mon.mean.nc

The above command will output much information on the computer screen, where we can find that each time slice has been seen as a single ***band***. We take the last band as an example. 

![img](img/gdalinfo.png)

It is worth noting there are two extra parameters of ***scale_factor*** and ***add_offset***. According to [NetCDF Attribute Conventions](http://www.bic.mni.mcgill.ca/users/sean/Docs/netcdf/guide.txn_18.html):

- scale_factor

> If present for a variable, the data are to be multiplied by this factor after the data are read by the application that accesses the data.

- add_offset

> If present for a variable, this number is to be added to the data after it is read by the application that accesses the data. If both scale_factor and add_offset attributes are present, the data are first scaled before the offset is added. The attributes scale_factor and add_offset can be used together to provide simple data compression to store low-resolution floating-point data as small integers in a netCDF file. When scaled data are written, the application should first subtract the offset and then divide by the scale factor.

When scale_factor and add_offset are used for packing, the associated variable (containing the packed data) is typically of type byte or short, whereas the unpacked values are intended to be of type float or double. The attributes scale_factor and add_offset should both be of the type intended for the unpacked data, e.g. float or double.

# Convert NetCDF to GeoTiff image(s)

Keep in mind that the band number starts from ***1*** in gdal_translate. The command option of ***-unscale*** can be used to tell gdal_translate to apply the scale/offset metadata for the bands to convert scaled values to unscaled values. Then we can easily convert one band (or a time slice) of the NetCDF data into a geotiff format just use the command:

In [ ]:
gdal_translate -ot Float64 NETCDF:mslp.mon.mean.nc:mslp -b 1 -unscale "mslp_01.tif"

However, we'd like to export all time slices with meaningful names such as ***mslp_yyyy-mm.tif***. Now it is show time for CDO. We can get all date-time information of the data using

In [ ]:
cdo showdate mslp.mon.mean.nc

The output looks like
![dates](img/cdoshowdate.png)

Now I think you should already figure out how to finish the left task. Yes, put them into a bash script.

In [ ]:
#!/bin/bash

infile=mslp.mon.mean.nc
band=1
for idate in $(cdo showdate $infile) 
do
  echo $band
  ym="${idate:0:7}"
  gdal_translate -ot Float64 NETCDF:$infile:mslp -b $band -unscale "mslp_$ym.tif"
  ((band++))
done
echo All done

Done! Congratulations! Now you already get a new option to Export Each Time Slice From A NetCDF Data As A Single Raster.

![list](img/filelist.png)

If you like other methods, you can refer this [python tutorial](https://www.linkedin.com/pulse/convert-netcdf4-file-geotiff-using-python-chonghua-yin/).

## References

https://code.mpimet.mpg.de/projects/cdo/

https://gdal.org/programs/gdal_translate.html#gdal-translate

NCEP-DOE AMIP-II Reanalysis (R-2): M. Kanamitsu, W. Ebisuzaki, J. Woollen, S-K Yang, J.J. Hnilo, M. Fiorino, and G. L. Potter. 1631-1643, Nov 2002, Bulletin of the American Meteorological Society.